In [196]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')

# 전처리

In [197]:
# column 삭제
def delete_col(df) :
    del_list = ['title','dir_prev_bfnum' ,'director']  # 'num_staff','num_actor','dir_prev_num'
    for i in del_list :
        del df[i]
    return df


# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

# 배급사 mapping
def distributor(df, distributor_mapper) :
    return make_col(df, 'distributor', distributor_mapper )
    

# genre mapping
def genre(df) :
    df = pd.get_dummies(df, columns=['genre'], prefix='genre')
    return df

# release time
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(a[2])
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

# time 
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

# screen_rat
# def screen_rat(df) :
#     tmp = sorted(df['screening_rat'].unique(), reverse=False)

#     mapping = {}
#     num = 1
#     for i in tmp :
#         mapping.update({i : num})
#         num += 1

#     return make_col(df, 'screening_rat', mapping)

def screen_rat(df) :
    df = pd.get_dummies(df, columns=['screening_rat'], prefix='genre')
    return df

In [198]:
# 배급사 mapper는 정해둔다
distributor_mapper = df['distributor'].value_counts()
distributor_mapper.value_counts()

mapping = {}
for i in range(len(distributor_mapper)):

    if distributor_mapper[i] >= 30 :
        mapping.update({ distributor_mapper.index[i] : 4 })

    elif distributor_mapper[i] >= 10 :
        mapping.update({ distributor_mapper.index[i] : 3 })
        
    elif distributor_mapper[i] >= 2 :
        mapping.update({ distributor_mapper.index[i] : 2 })
        
    else :
        mapping.update({ distributor_mapper.index[i] : 1 })
        
# genre_all = df['genre'].unique()

# mapping_genre = {}
# num = 2
# for genre in genre_all :
#     mapping.update({genre : num})
#     num += 1

In [199]:
df = delete_col(df)
df = distributor(df, mapping)
df = genre(df)
df = release_time(df)
df = time(df)
df = screen_rat(df)

df

,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,...,genre_액션,genre_코미디,release_year,release_month,release_day,new_time,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
0,0,91,2,23398,4,0,0,0,0,0,...,1,0,2012,11,3,2,0,0,0,1
1,2,387,3,7072501,3,0,0,1,0,0,...,0,0,2015,11,2,3,0,0,0,1
2,4,343,4,6959083,3,0,0,0,0,0,...,1,0,2013,6,1,2,0,1,0,0
3,2,20,6,217866,4,0,0,0,0,0,...,0,1,2012,7,2,2,0,0,1,0
4,1,251,2,483387,2,0,0,0,0,0,...,0,1,2010,11,1,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1,510,7,1475091,4,0,0,0,0,1,...,0,0,2014,8,2,2,0,0,0,1
596,1,286,6,1716438,3,0,0,0,0,1,...,0,0,2013,3,2,2,0,1,0,0
597,0,123,4,2475,3,0,1,0,0,0,...,0,0,2010,9,3,2,0,0,0,1
598,0,431,4,2192525,4,0,0,1,0,0,...,0,0,2015,5,2,2,0,1,0,0


In [200]:
testdf = delete_col(testdf)
testdf = distributor(testdf, mapping)
testdf = genre(testdf)
testdf = release_time(testdf)
testdf = time(testdf)
testdf = screen_rat(testdf)

testdf

,dir_prev_num,num_staff,num_actor,new_distributor,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,genre_액션,genre_코미디,release_year,release_month,release_day,new_time,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
0,2,304,3,2,0,0,1,0,0,0,...,0,0,2010,1,1,2,0,0,0,1
1,4,275,3,3,0,0,0,0,0,1,...,0,0,2010,1,2,2,1,0,0,0
2,3,419,7,4,0,0,0,0,1,0,...,0,0,2010,1,3,2,1,0,0,0
3,2,408,2,3,0,0,0,0,0,0,...,1,0,2010,2,1,2,0,1,0,0
4,1,380,1,4,0,1,0,0,0,0,...,0,0,2010,2,2,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,1,4,4,1,0,0,0,0,1,0,...,0,0,2015,11,2,2,0,1,0,0
239,0,18,2,3,0,0,0,1,0,0,...,0,0,2015,10,3,2,1,0,0,0
240,0,224,4,1,0,0,1,0,0,0,...,0,0,2015,10,3,2,0,0,0,1
241,1,68,7,1,0,0,0,0,1,0,...,0,0,2015,10,3,2,0,0,0,1


In [188]:
df.corr().sort_values(by='box_off_num', ascending = False)

,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,...,genre_액션,genre_코미디,release_year,release_month,release_day,new_time,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
box_off_num,0.259674,0.544265,0.111179,1.000000,0.297959,0.088009,-0.017627,0.184869,-0.150310,-0.034488,...,0.181191,0.082780,-0.002497,0.019104,-0.011023,0.294977,0.033896,0.210368,-0.133525,-0.135180
num_staff,0.450706,1.000000,0.077871,0.544265,0.418679,0.041529,0.041874,0.209779,-0.344846,0.061664,...,0.249187,0.108996,-0.032891,-0.002841,0.008254,0.329142,0.060474,0.246535,-0.273736,-0.085689
new_distributor,0.230853,0.418679,0.040620,0.297959,1.000000,0.029575,0.020508,0.076512,-0.129674,-0.004181,...,0.094063,0.034381,-0.055365,-0.005287,-0.066614,0.134728,0.005224,0.143713,-0.095909,-0.074560
new_time,0.160672,0.329142,0.033701,0.294977,0.134728,0.028842,-0.018682,0.176828,-0.107562,0.030352,...,0.119450,-0.021196,-0.052543,0.008131,0.029267,1.000000,-0.030817,0.047551,-0.076225,0.034978
dir_prev_num,1.000000,0.450706,0.014006,0.259674,0.230853,0.054248,0.050714,0.077037,-0.173434,-0.040161,...,0.183462,0.057303,0.132621,0.034019,0.030678,0.160672,0.020939,0.196600,-0.166861,-0.085814
genre_15세 관람가,0.196600,0.246535,0.065280,0.210368,0.143713,0.039323,0.067178,-0.001531,-0.139453,0.011674,...,0.109911,0.126227,-0.071860,-0.051320,-0.018937,0.047551,-0.322418,1.000000,-0.303177,-0.511330
genre_느와르,0.077037,0.209779,-0.010128,0.184869,0.076512,-0.032304,-0.059554,1.000000,-0.092970,-0.165761,...,-0.048027,-0.067569,0.013021,-0.033931,0.008163,0.176828,-0.098240,-0.001531,-0.092378,0.149692
genre_액션,0.183462,0.249187,-0.009006,0.181191,0.094063,-0.032926,-0.060700,-0.048027,-0.094759,-0.168950,...,1.000000,-0.068869,-0.013522,0.031853,0.075406,0.119450,-0.079095,0.109911,-0.094155,0.024687
num_actor,0.014006,0.077871,1.000000,0.111179,0.040620,-0.000874,0.019558,-0.010128,-0.076703,-0.018618,...,-0.009006,0.131034,-0.098869,-0.015063,0.009580,0.033701,-0.018293,0.065280,0.017003,-0.063550
genre_SF,0.054248,0.041529,-0.000874,0.088009,0.029575,1.000000,-0.040828,-0.032304,-0.063737,-0.113639,...,-0.032926,-0.046323,0.021702,-0.007262,-0.028737,0.028842,0.024075,0.039323,-0.031559,-0.034315


In [201]:
del df['release_year']
del testdf['release_year']

# 모델링

In [190]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [85]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

## 사이킷런

In [139]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

In [119]:
save_file(prediction, 1)
# 1744049.2329963981	

done


In [184]:
save_file(prediction, 4)
# 1509223.1428086378	 , 'num_staff','num_actor'를 추가하였을 때

done


In [15]:
save_file(prediction, 6)

done


In [28]:
save_file(prediction, 7)
# 1661234.9589730736	

done


In [39]:
save_file(prediction, 8)
# 1507395.893538275	, 'num_staff','num_actor'를 추가하였을 때 , 상영시간 -> get_dummies

done


In [47]:
save_file(prediction, 9)
# 1507395.893538275	 num_staff','num_actor'를 추가하였을 때 , 

# 위와 결과가 같기 때문에 get_dummies와는 큰 관계가 없다

done


In [ ]:
save_file(prediction, 11)
# 1507395.893538275	, 'num_staff','num_actor'를 추가하였을 때
# 상영시간 -> get_dummies
# times 원래 데이터 사용

# 2651221.073888509	

In [87]:
save_file(prediction, 12)
# 1507395.893538275	, 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# 상영시간 -> get_dummies

# 1506004.9532714835

done


In [144]:
prediction = abs(prediction.astype(int))
# 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# release_year 제거
# abs( int )

save_file(prediction, 13)
# 1504505.219927887	

done


## tensorflow

In [145]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(shape = [None,24], dtype=tf.float32)
Y = tf.placeholder(shape = [None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([24,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

H = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(H-Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000002)
train = optimizer.minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(500000) :
    _, cost_val = sess.run([train, cost],
                          feed_dict = {
                              X : train_X.values,
                              Y : train_Y.values.reshape(-1,1)
                          })
    if step % 50000 == 0 :
        print('cost값은 : {}'.format(cost_val))
    
prediction = sess.run(H, feed_dict = {X : test_X.values, })

cost값은 : 3840678821888.0
cost값은 : 2342387449856.0
cost값은 : 2337892466688.0
cost값은 : 2334118379520.0
cost값은 : 2330900299776.0
cost값은 : 2328118689792.0
cost값은 : 2325680488448.0
cost값은 : 2323510984704.0
cost값은 : 2321561419776.0
cost값은 : 2319772811264.0


In [172]:
save_file(prediction, 2)
# learning_rate = 0.0000001
# 30000번 학습
# 	1857916.1853515022	 

done


In [176]:
save_file(prediction, 3)
# 300000번 학습
# 1844690.7292044302	

done


In [186]:
save_file(prediction, 5)
# num_staff','num_actor'를 추가하였을 때 
# 	1493283.9481655352	

done


In [55]:
save_file(prediction, 10)
# num_staff','num_actor'를 추가하였을 때 
# genre, screening_rat : get_dummies
# 1493679.4503123555	

done


In [ ]:
save_file(prediction, 11)
#1496128.6506115296	

In [162]:
prediction = abs(prediction.astype(int))
# 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# release_year 제거
# -값들을 0으로 변경

save_file(prediction, 14)

# 1492642.1951272835

done


In [ ]:
n = 0
a = 0
idx = []
for i in range(len(prediction)) :
    if prediction[i] < 0 :
        prediction[i] = 0
        n += 1
        #print(i, a)
        idx.append(a)
    a += 1
#test_X.loc[idx, :].sort_values(by = 'num_staff', ascending=False)
prediction

# 3. k-fold , 

In [204]:
df.columns = ['dir_prev_num', 'num_staff', 'num_actor', 'box_off_num',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy', 'release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']
testdf.columns = ['dir_prev_num', 'num_staff', 'num_actor',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy', 'release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']

In [173]:
!pip install lightgbm

In [220]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMRegressor(random_state = 777, n_estimators=1000)
features = ['dir_prev_num', 'num_staff', 'num_actor',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy', 'release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']

target = ['box_off_num']

params = {
    'learning_rate' : [0.1, 0.01, 0.003],
    'min_child_samples' : [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid = params,
                 scoring='neg_mean_squared_error',
                 cv=k_fold)

X_train, X_test, Y_train = df[features], testdf[features], df[target]


In [221]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=1000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                    

In [224]:
gs.best_params_

{'learning_rate': 0.003, 'min_child_samples': 20}

In [226]:
model = lgb.LGBMRegressor(random_state = 777, n_estimators=1000, learning_rate= 0.003, min_child_samples= 20)

In [227]:
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5, shuffle=True, random_state = 777)

models = []
for train_idx, val_idx in k_fold.split(X_train) :
    x_t = X_train.iloc[train_idx]
    y_t = Y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set = (x_val, y_val), early_stopping_rounds=100, verbose=100))
    

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.58628e+12
[200]	valid_0's l2: 2.50961e+12
[300]	valid_0's l2: 2.44963e+12
[400]	valid_0's l2: 2.39206e+12
[500]	valid_0's l2: 2.35724e+12
[600]	valid_0's l2: 2.34925e+12
[700]	valid_0's l2: 2.35439e+12
Early stopping, best iteration is:
[626]	valid_0's l2: 2.34791e+12
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.19663e+12
[200]	valid_0's l2: 4.88573e+12
[300]	valid_0's l2: 4.67607e+12
[400]	valid_0's l2: 4.54543e+12
[500]	valid_0's l2: 4.48937e+12
[600]	valid_0's l2: 4.47264e+12
[700]	valid_0's l2: 4.4517e+12
[800]	valid_0's l2: 4.43181e+12
[900]	valid_0's l2: 4.41575e+12
[1000]	valid_0's l2: 4.39768e+12
Did not meet early stopping. Best iteration is:
[990]	valid_0's l2: 4.39689e+12
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.45317e+12
[200]	valid_0's l2: 3.24459e+12
[300]	valid_0's l2: 3.15623e+12
[400]	valid_0's l2: 3.0863e

In [228]:
preds = []
for model in models :
    preds.append(model.predict(X_test))
len(preds)

5

In [229]:
import numpy as np
prediction = np.mean(preds, axis=0)

In [231]:
save_file(prediction, 15)

done


In [232]:
prediction

array([1468188.27600174,  772256.05748746, 1747565.93306415,
       1796729.42448321, 1518192.47806996,  246157.89770498,
        195851.33521065,  142771.16854361,  260064.46818421,
        808029.10798224,  298189.9711133 , 1552902.43087113,
       1689547.98312557,  293841.03868722, 1779265.8619409 ,
       3169339.13760333,  294017.8895555 , 1240946.149739  ,
        553905.49945265, 1180557.1263371 ,  166113.32538256,
        224567.34411964,  326141.50841903,  604587.16083079,
        123974.24018805, 2088555.48291001,  112821.7277402 ,
       1472254.91292489,  108408.96791867,  654262.58311402,
       1102920.17264877,  214843.86712263,  415025.11357761,
        918262.63762453,  193389.79404135,  201642.61812722,
        216152.58325555,  269511.64247743,  178023.1778155 ,
        143156.94238249,  606980.26978928,  118036.13176761,
        197586.06372152,  446901.82700344, 1005423.91092728,
        919113.82132085,  219177.93605117,  128871.85898303,
        663701.69678743,